In [1]:
import numpy as np
import pandas as pd
from random import sample

import hyperopt
from hyperopt import hp, Trials
from hyperopt.pyll.base import scope

from sklearn.metrics import f1_score, log_loss, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

from coremdlr.models import NetworkModel
from coremdlr.config import defaults

from coremdlr.datasets import WellLoader, FaciesDataset
from coremdlr.datasets.utils import infer_test_wells

import matplotlib.pyplot as plt
%matplotlib inline

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ross/anaconda3/envs/core-dev/lib/python3

In [2]:
BCNN_SEARCH_SPACE = {
    'feature' : 'image',
    'network' : 'bilinear_cnn',
    'summary' : False,
    'sequence_size' : 12,
    'network_args' : {
        'backbone_cnn' : 'vgg16',
        'conv1x1' : scope.int(hp.quniform('conv1x1', 16, 32, 8)),
        'apply_rowwise' : True,
        'lstm_features' : None,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam',
        'clipnorm' : 10.
    }
}

IMAGE_ARGS = {
    'image_width' : 200,
    'crop_method' : 'center',
}


fit_args = {
    'batch_size' : 4,
    'epochs' : 15,
    'class_weighted' : True
}

CHKPT_FNAME = 'temp_best_weights.h5'

In [3]:
def run_experiment(num_train, max_evals=15, search_space=BCNN_SEARCH_SPACE):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    #datasets = [FaciesDataset(s, infer_test_wells(s), features=['image'],
    #                          image_args=IMAGE_ARGS) for s in train_sets[:2]]
    
    #for dset in datasets:
    #    dset.load_or_generate_data()
    
    def train_model(model_config):
        
        acc_scores, f1_scores, log_losses = [], [], []
        
        for i, train_set in enumerate(train_sets):
            dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['image'],
                                image_args=IMAGE_ARGS)
        
            dset.load_or_generate_data()
            
            model = NetworkModel(dset, model_args=model_config)
            
            chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
            earlystop_callback = EarlyStopping(monitor='val_loss', patience=3)
            fit_args['callbacks'] = [chkpt_callback, earlystop_callback]
            
            print(f'Training with config: {model_config}')
            
            model.fit(dset, **fit_args)
            
            model.network.load_weights(CHKPT_FNAME)
            
            y_proba = model.predict_proba(dset.X_test)
            y_pred = np.argmax(y_proba, -1)
            
            acc_scores.append(accuracy_score(dset.y_test, y_pred))
            
            f1_scores.append(f1_score(dset.y_test, y_pred, average='macro'))
            
            log_losses.append(log_loss(dset.y_test, y_proba, 
                                      sample_weight=compute_sample_weight('balanced', dset.y_test)))
            
            del dset
            del model
            print("GOT HERE !!!!!!!!!!!!!!!!!!!!")
            
        return {'loss' : np.mean(log_losses),
                'acc_scores' : acc_scores,
                'f1_scores' : f1_scores,
                'log_losses' : log_losses,
                'status' : hyperopt.STATUS_OK}
    
    
    trials = Trials()
    
    best_params = hyperopt.fmin(
        fn=train_model,
        space=BCNN_SEARCH_SPACE,
        algo=hyperopt.rand.suggest,
        max_evals=max_evals,
        trials=trials
    )
    
    return trials, best_params

In [4]:
trails3, best_params3 = run_experiment(3, max_evals=3)

Loading Well:                                      
204-24a-7                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (19294,)), ('top', (19294,)), ('base', (19294,)), ('image', (19294, 32, 200, 3))]
Loading Well:                                      
204-24a-6                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (13034,)), ('top', (13034,)), ('base', (13034,)), ('image', (13034, 32, 200, 3))]
Loading Well:                                      
204-20-3                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (10906,)), ('top', (109

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)



Loading Well:                                      
205-21b-3                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('image', (3881, 32, 200, 3))]
Loading Well:                                      
204-19-6                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (1981,)), ('top', (1981,)), ('base', (1981,)), ('image', (1981, 32, 200, 3))]
Loading Well:                                      
204-20-1                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (11210,)), ('top', (11210,)), (

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Training with config: {'feature': 'image', 'network': 'bilinear_cnn', 'network_args': {'apply_rowwise': True, 'backbone_cnn': 'vgg16', 'conv1x1': 24, 'dropout_rate': 0.5, 'lstm_features': None}, 'optimizer_args': {'clipnorm': 10.0}, 'sequence_size': 12, 'summary': False}
Shapes of `(batch_X, batch_y)`: (4, 384, 200, 3), (4, 12, 6)
Epoch 1/15                                         
  0%|          | 0/3 [01:27<?, ?it/s, best loss: ?]WARNING:tensorflow:From /home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
   1/1800 [..............................]         
 - ETA: 2:20:37 - loss: 1.4843 - acc: 0.

...

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)



Feature shapes:                                    
[('depth', (9598,)), ('top', (9598,)), ('base', (9598,)), ('image', (9598, 32, 200, 3))]
Loading Well:                                      
204-19-7                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (9807,)), ('top', (9807,)), ('base', (9807,)), ('image', (9807, 32, 200, 3))]
Loading Well:                                      
205-21b-3                                          
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (3881,)), ('top', (3881,)), ('base', (3881,)), ('image', (3881, 32, 200, 3))]
  0%|          | 0/3 [40:00<?, ?it/s, best loss: ?]

/home/ross/anaconda3/envs/core-dev/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)



Loading Well:                                      
204-20-3                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (10906,)), ('top', (10906,)), ('base', (10906,)), ('image', (10906, 32, 200, 3))]
Loading Well:                                      
204-19-6                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (1981,)), ('top', (1981,)), ('base', (1981,)), ('image', (1981, 32, 200, 3))]
Loading Well:                                      
204-20-1                                           
 from                                              
/home/ross/Dropbox/core_data/facies/train_data     
Feature shapes:                                    
[('depth', (11210,)), ('top', (11210,)

MemoryError: Unable to allocate array with shape (67941, 32, 200, 3) and data type float64

In [ ]:
trails3.results

In [ ]:
best_params3

In [ ]:
trails3.best_trial

In [ ]:
def pred_results_df(num_train, model_config):
    
    assert num_train in [3, 6, 9], 'Must use one of predefined train/test splits'
    
    train_sets = getattr(defaults, f'TRAIN_SETS_{num_train}')
    
    pred_dfs = []
    
    for i, train_set in enumerate(train_sets):
        dset = FaciesDataset(train_set, infer_test_wells(train_set), features=['pseudoGR'], 
                             pseudoGR_args={'features' : ['mean', 'var'], 'per_channel' : True})
        
        dset.load_or_generate_data()
        
        model = NetworkModel(dset, model_args=model_config)
        
        chkpt_callback = ModelCheckpoint(CHKPT_FNAME, monitor='val_loss', verbose=1,
                                        save_best_only=True, 
                                        save_weights_only=True)
        
        model.fit(dset, callbacks=[chkpt_callback], verbose=False, **fit_args)
        
        model.network.load_weights(CHKPT_FNAME)
        
        #print('predict_proba: ', model.predict_proba(dset.X_test))
        
        for test_well_name in dset.test_well_names:
            df = model.preds_dataframe(test_well_name)
            df['well'] = test_well_name
            df['set_number'] = i
            pred_dfs.append(df)
        
    return pd.concat(pred_dfs)

In [ ]:
params = {
    'feature' : 'pseudoGR',
    'network' : 'wavenet',
    'summary' : False,
    'sequence_size' : 128,
    'network_args' : {
        'num_blocks' : 8,
        'block_filters' : 208,
        'residual_filters' : 40,
        'output_resolution' : 32,
        'dropout_rate' : 0.5,
    },
    'optimizer_args' : {
        'optimizer' : 'Adam'
    }
}

preds_df_3 = pred_results_df(3, params)
preds_df_3.to_csv('pgr_wavenet_results_train_sets3.csv')

In [ ]:
accuracy_score(preds_df_3.y_pred.values, preds_df_3.y_true.values)